In [1]:
## Load helpers

# Python Basic Tools
import pandas as pd
import numpy as np
import seaborn as sn
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# Sklearn Basic Tools
import sklearn.metrics
import sklearn.neighbors
from sklearn.model_selection import train_test_split

# Sklearn Regression and Classification Tools
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.naive_bayes import GaussianNB

# Usual cross-val tools and Grid searching 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

from scipy.stats import norm, zscore
# Set random seed
np.random.seed(47)


In [2]:
# Import Combined Dataset
## Include Monthly Index Return data, Macro Indicators, Valuation Indicators and Momentum Indicators added
## Already Normorlized
## Already Check Stationarity, then convert non-ststionay to ststionary

df=  pd.read_csv("All Data.csv",parse_dates=['Date'])
df


,Date,Unemployment Rate,New Privately-Owned Housing Units Started: Total Units,10-Year Treasury Constant Maturity Minus Federal Funds Rate,Producer Price Index by Commodity: All Commodities,Producer Price Index - Petroleum,"All Employees, Total Nonfarm",M1 for the United States,PMI Composite Index,Consumer Price Index: Total All Items for the United States,...,DJIA All Caps _lag_3,DJIA All Caps _lag_4,DJIA All Caps _lag_5,DJIA All Caps _lag_6,DJIA All Caps _lag_7,DJIA All Caps _lag_8,DJIA All Caps _lag_9,DJIA All Caps _lag_10,DJIA All Caps _lag_11,DJIA All Caps _lag_12
0,2000-05-01,-1.185698,-0.590086,0.949817,0.245481,0.683491,-0.512409,-1.822092,-0.934958,-0.177245,...,0.117650,-0.199800,0.335000,0.160714,0.303524,-0.119952,-0.037864,-0.150667,0.234545,-0.104150
1,2000-06-01,-1.205172,-0.167572,-2.427361,1.094435,0.720068,-0.534268,-1.043800,-1.245555,0.949817,...,0.264565,0.117650,-0.199800,0.335000,0.160714,0.303524,-0.119952,-0.037864,-0.150667,0.234545
2,2000-07-01,-1.225114,-1.011479,-0.206370,-0.448223,-0.659608,-0.469401,-0.578780,0.647806,0.177245,...,-0.253474,0.264565,0.117650,-0.199800,0.335000,0.160714,0.303524,-0.119952,-0.037864,-0.150667
3,2000-08-01,-1.094435,0.836201,-0.891570,-0.757636,0.314881,-0.480068,-1.129813,-1.580168,-0.590086,...,-0.147818,-0.253474,0.264565,0.117650,-0.199800,0.335000,0.160714,0.303524,-0.119952,-0.037864
4,2000-09-01,-1.432184,-0.355208,-0.157914,0.822757,0.480068,-0.437709,-0.849798,-0.090668,0.934958,...,0.202455,-0.147818,-0.253474,0.264565,0.117650,-0.199800,0.335000,0.160714,0.303524,-0.119952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2020-12-01,0.601468,1.225114,0.375586,1.060376,1.077249,0.744997,0.905843,1.459285,-0.245481,...,-0.166333,0.332143,0.253773,0.121227,0.273150,0.628810,-0.503818,-0.437000,-0.003333,0.136190
248,2021-01-01,0.545294,-0.375586,0.744997,1.774106,1.094435,0.783284,1.288079,-1.225114,0.624469,...,-0.089682,-0.166333,0.332143,0.253773,0.121227,0.273150,0.628810,-0.503818,-0.437000,-0.003333
249,2021-02-01,0.501571,-1.822092,0.849798,1.998824,1.111950,0.849798,1.356611,1.185698,1.094435,...,0.582850,-0.089682,-0.166333,0.332143,0.253773,0.121227,0.273150,0.628810,-0.503818,-0.437000
250,2021-03-01,0.375586,4.000000,1.356611,4.000000,1.266531,0.920303,1.245555,2.164601,1.580168,...,0.201000,0.582850,-0.089682,-0.166333,0.332143,0.253773,0.121227,0.273150,0.628810,-0.503818


In [3]:
# y
df1=df[df.columns[45:56]]
df1

,Market,Momentum,Quality,Min Vol,Value,Aggressive Growth (QQQ),Russell Large,Russell Mid,Russell Small,Russell All Caps,DJIA All Caps
0,-0.082591,-0.266727,-0.058182,-0.037409,-0.057727,-0.489636,-0.106955,-0.103000,-0.250500,-0.116455,-0.147818
1,0.116000,0.276955,0.185045,-0.027273,-0.119091,0.565136,0.113818,0.143227,0.394818,0.138545,0.202455
2,-0.073600,-0.077100,-0.043000,-0.052350,0.043550,-0.161050,-0.085650,-0.050100,-0.153750,-0.083350,-0.096250
3,0.264304,0.343522,0.179913,0.222174,0.314087,0.576391,0.287609,0.400739,0.322435,0.313217,0.307000
4,-0.267200,-0.478450,-0.149000,-0.017700,-0.435300,-0.662650,-0.281950,-0.066900,-0.141350,-0.226950,-0.234050
...,...,...,...,...,...,...,...,...,...,...,...
247,0.173000,0.158864,0.168045,0.105909,0.136818,0.220364,0.182773,0.210318,0.382773,0.201545,0.201000
248,-0.048000,0.098526,-0.144947,-0.139053,0.179316,0.023474,-0.048421,-0.007000,0.270368,-0.017842,-0.011684
249,0.147000,-0.018842,0.174842,-0.018053,0.360684,0.001526,0.104895,0.291263,0.331421,0.166579,0.170684
250,0.191739,-0.028783,0.209696,0.237826,0.310870,0.090696,0.184261,0.124087,0.063826,0.159435,0.155435


In [4]:
# X
df2 = df.drop(columns=list(df1.columns))
df2= df2[df2.columns[1:704]]
df2

,Unemployment Rate,New Privately-Owned Housing Units Started: Total Units,10-Year Treasury Constant Maturity Minus Federal Funds Rate,Producer Price Index by Commodity: All Commodities,Producer Price Index - Petroleum,"All Employees, Total Nonfarm",M1 for the United States,PMI Composite Index,Consumer Price Index: Total All Items for the United States,Value-Weighted \nReturn-incl. dividends,...,DJIA All Caps _lag_2,DJIA All Caps _lag_3,DJIA All Caps _lag_4,DJIA All Caps _lag_5,DJIA All Caps _lag_6,DJIA All Caps _lag_7,DJIA All Caps _lag_8,DJIA All Caps _lag_9,DJIA All Caps _lag_10,DJIA All Caps _lag_11
0,-1.185698,-0.590086,0.949817,0.245481,0.683491,-0.512409,-1.822092,-0.934958,-0.177245,0.809459,...,0.264565,0.117650,-0.199800,0.335000,0.160714,0.303524,-0.119952,-0.037864,-0.150667,0.234545
1,-1.205172,-0.167572,-2.427361,1.094435,0.720068,-0.534268,-1.043800,-1.245555,0.949817,1.614293,...,-0.253474,0.264565,0.117650,-0.199800,0.335000,0.160714,0.303524,-0.119952,-0.037864,-0.150667
2,-1.225114,-1.011479,-0.206370,-0.448223,-0.659608,-0.469401,-0.578780,0.647806,0.177245,0.707771,...,-0.147818,-0.253474,0.264565,0.117650,-0.199800,0.335000,0.160714,0.303524,-0.119952,-0.037864
3,-1.094435,0.836201,-0.891570,-0.757636,0.314881,-0.480068,-1.129813,-1.580168,-0.590086,0.186936,...,0.202455,-0.147818,-0.253474,0.264565,0.117650,-0.199800,0.335000,0.160714,0.303524,-0.119952
4,-1.432184,-0.355208,-0.157914,0.822757,0.480068,-0.437709,-0.849798,-0.090668,0.934958,-0.119420,...,-0.096250,0.202455,-0.147818,-0.253474,0.264565,0.117650,-0.199800,0.335000,0.160714,0.303524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0.601468,1.225114,0.375586,1.060376,1.077249,0.744997,0.905843,1.459285,-0.245481,0.949817,...,-0.089682,-0.166333,0.332143,0.253773,0.121227,0.273150,0.628810,-0.503818,-0.437000,-0.003333
248,0.545294,-0.375586,0.744997,1.774106,1.094435,0.783284,1.288079,-1.225114,0.624469,0.934958,...,0.582850,-0.089682,-0.166333,0.332143,0.253773,0.121227,0.273150,0.628810,-0.503818,-0.437000
249,0.501571,-1.822092,0.849798,1.998824,1.111950,0.849798,1.356611,1.185698,1.094435,0.920303,...,0.201000,0.582850,-0.089682,-0.166333,0.332143,0.253773,0.121227,0.273150,0.628810,-0.503818
250,0.375586,4.000000,1.356611,4.000000,1.266531,0.920303,1.245555,2.164601,1.580168,0.980182,...,-0.011684,0.201000,0.582850,-0.089682,-0.166333,0.332143,0.253773,0.121227,0.273150,0.628810


In [167]:
# Seperate data with features and labels
X = df2 #features
y =df1['Market']

# Split the dataframe into random train and test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2) #split by half

from sklearn.model_selection import GridSearchCV
param_grid = { 'bootstrap': [True,False],
              'max_depth': [5, 10], 
              'max_features': ['auto', 'log2'], 
              'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15], 
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10]}

rfr = RandomForestRegressor()

g_search = GridSearchCV(estimator = rfr, param_grid = param_grid,
                        cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

g_search.fit(X_train, y_train)
print(g_search.best_params_)
print(g_search.best_score_)

g_search.fit(X_test, y_test)
print(g_search.best_score_)



{'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 11}
0.1110094489714722
0.32363531130205386


In [162]:
# Seperate data with features and labels
X = df2 #features
y =df1['Momentum']

# Split the dataframe into random train and test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=47) #split by half

from sklearn.model_selection import GridSearchCV
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

rfr = RandomForestRegressor(random_state = 1)

g_search = GridSearchCV(estimator = rfr, param_grid = param_grid,
                        cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

g_search.fit(X_train, y_train)
print(g_search.best_params_)
print(g_search.best_score_)

g_search.fit(X_test, y_test)
print(g_search.best_score_)

{'bootstrap': True, 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 7}
-0.03374657554902324
0.32598222306252195


In [154]:
# Seperate data with features and labels
X = df2 #features
y =df1['Quality']

# Split the dataframe into random train and test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=47) #split by half

from sklearn.model_selection import GridSearchCV
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

rfr = RandomForestRegressor(random_state = 1)

g_search = GridSearchCV(estimator = rfr, param_grid = param_grid,
                        cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

g_search.fit(X_train, y_train)
print(g_search.best_params_)
print(g_search.best_score_)

g_search.fit(X_test, y_test)
print(g_search.best_score_)

{'bootstrap': True, 'max_depth': 5, 'max_features': 'auto', 'n_estimators': 12}
-0.065967795145032
0.18507225374372427


In [155]:
# Seperate data with features and labels
X = df2 #features
y =df1['Min Vol']

# Split the dataframe into random train and test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=47) #split by half

from sklearn.model_selection import GridSearchCV
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

rfr = RandomForestRegressor(random_state = 1)

g_search = GridSearchCV(estimator = rfr, param_grid = param_grid,
                        cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

g_search.fit(X_train, y_train)
print(g_search.best_params_)
print(g_search.best_score_)

g_search.fit(X_test, y_test)
print(g_search.best_score_)

{'bootstrap': True, 'max_depth': 5, 'max_features': 'auto', 'n_estimators': 12}
-0.0824664765616423
0.19940189591667598


In [156]:
# Seperate data with features and labels
X = df2 #features
y =df1['Value']

# Split the dataframe into random train and test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=47) #split by half

from sklearn.model_selection import GridSearchCV
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

rfr = RandomForestRegressor(random_state = 1)

g_search = GridSearchCV(estimator = rfr, param_grid = param_grid,
                        cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

g_search.fit(X_train, y_train)
print(g_search.best_params_)
print(g_search.best_score_)

g_search.fit(X_test, y_test)
print(g_search.best_score_)


{'bootstrap': True, 'max_depth': 5, 'max_features': 'auto', 'n_estimators': 15}
0.039582690043864645
0.1693502100660261


In [157]:
# Seperate data with features and labels
X = df2 #features
y =df1['Russell Large']

# Split the dataframe into random train and test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=47) #split by half

from sklearn.model_selection import GridSearchCV
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

rfr = RandomForestRegressor(random_state = 1)

g_search = GridSearchCV(estimator = rfr, param_grid = param_grid,
                        cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

g_search.fit(X_train, y_train)
print(g_search.best_params_)
print(g_search.best_score_)

g_search.fit(X_test, y_test)
print(g_search.best_score_)

{'bootstrap': True, 'max_depth': 5, 'max_features': 'auto', 'n_estimators': 15}
0.04454686048241652
0.239835174167643


In [158]:
# Seperate data with features and labels
X = df2 #features
y =df1['Russell Mid']

# Split the dataframe into random train and test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=47) #split by half

from sklearn.model_selection import GridSearchCV
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

rfr = RandomForestRegressor(random_state = 1)

g_search = GridSearchCV(estimator = rfr, param_grid = param_grid,
                        cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

g_search.fit(X_train, y_train)
print(g_search.best_params_)
print(g_search.best_score_)

g_search.fit(X_test, y_test)
print(g_search.best_score_)

{'bootstrap': True, 'max_depth': 5, 'max_features': 'auto', 'n_estimators': 15}
0.04036464768762127
0.31747703423003715


In [159]:
# Seperate data with features and labels
X = df2 #features
y =df1['Russell Small']

# Split the dataframe into random train and test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=47) #split by half

from sklearn.model_selection import GridSearchCV
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

rfr = RandomForestRegressor(random_state = 1)

g_search = GridSearchCV(estimator = rfr, param_grid = param_grid,
                        cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

g_search.fit(X_train, y_train)
print(g_search.best_params_)
print(g_search.best_score_)

g_search.fit(X_test, y_test)
print(g_search.best_score_)

{'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'n_estimators': 15}
0.05494271052435203
0.248386893792265


In [160]:
# Seperate data with features and labels
X = df2 #features
y =df1['DJIA All Caps ']

# Split the dataframe into random train and test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=47) #split by half

from sklearn.model_selection import GridSearchCV
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

rfr = RandomForestRegressor(random_state = 1)

g_search = GridSearchCV(estimator = rfr, param_grid = param_grid,
                        cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

g_search.fit(X_train, y_train)
print(g_search.best_params_)
print(g_search.best_score_)

g_search.fit(X_test, y_test)
print(g_search.best_score_)


{'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'n_estimators': 15}
0.11277878807370623
0.3280702664487117


In [161]:
# Seperate data with features and labels
X = df2 #features
y =df1['Russell All Caps ']

# Split the dataframe into random train and test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=47) #split by half

from sklearn.model_selection import GridSearchCV
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

rfr = RandomForestRegressor(random_state = 1)

g_search = GridSearchCV(estimator = rfr, param_grid = param_grid,
                        cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

g_search.fit(X_train, y_train)
print(g_search.best_params_)
print(g_search.best_score_)

g_search.fit(X_test, y_test)
print(g_search.best_score_)

{'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'n_estimators': 15}
0.08678957882097198
0.33327207952094656
